In [2]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='PDA_SOD'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 8-17,23-23 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)


In [15]:
# from utils.df_handle import *

In [12]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y-%m-%d")

In [4]:
from_tb = "OM_PDASalesOrdDet"

In [7]:
table_name = "sync_dms_pda_sod"
table_temp = "sync_dms_pda_sod_temp"

In [18]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID,OrderNbr,LineRef) as pk,
BranchID,
OrderNbr,
LineRef,
InvtID,
LineQty,
OrderType,
OrigOrderNbr,
SiteID,
SlsPrice,
Crtd_Prog,
Crtd_User,
Crtd_Datetime,
LUpd_Datetime,
SlsperID,
BeforeVATPrice,
BeforeVATAmount,
AfterVATPrice,
AfterVATAmount,
VATAmount
from {from_tb}
where LUpd_DateTime >= @from
"""

In [19]:
print(usql)


DECLARE @from DATE = '2022-05-14'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
OrderNbr,
LineRef,
InvtID,
LineQty,
OrderType,
OrigOrderNbr,
SiteID,
SlsPrice,
Crtd_Prog,
Crtd_User,
Crtd_Datetime,
LUpd_Datetime,
SlsperID,
BeforeVATPrice,
BeforeVATAmount,
AfterVATPrice,
AfterVATAmount,
VATAmount
from OM_PDASalesOrdDet
where LUpd_DateTime >= @from
and Crtd_DateTime < @from



In [ ]:
table_name = "sync_dms_pda_sod"
table_temp = "sync_dms_pda_sod_temp"

In [5]:
#INSERT
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
# bq_values_insert(df, f"{table_temp}", 3)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_name};
# CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
# PARTITION BY DATE(crtd_datetime)
# CLUSTER BY pk,branchid,ordernbr,slsperid
# """
# execute_bq_query(sql)
dsql = \
f"""
delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow}'
"""
print("delete_sql: ", dsql)
execute_bq_query(dsql)
bq_values_insert(df, f"{table_name}", 2)

In [13]:
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_temp};
# """
# execute_bq_query(sql)

In [21]:
from utils.df_handle import *

In [23]:
#UPDATE
df_update = get_ms_df(usql)
df_update.columns = lower_col(df_update)
# df_update.columns
df_update['crtd_datetime'] = df_update.crtd_datetime.dt.normalize()
df_update1 = df_update['crtd_datetime']
df_update1.drop_duplicates(inplace=True)
tpl_dt = tuple(df_update1.dt.strftime('%Y-%m-%d').to_list()) + ('1900-01-01',)
# tpl_dt
df_update1 = df_update['pk']
df_update1.drop_duplicates(inplace=True)
tpl_pk = tuple(df_update1.to_list()) + ('',)
del_sql = \
f"""
DELETE FROM biteam.{table_name}
WHERE
DATE(crtd_datetime) in {tpl_dt}
AND concat(branchid,ordernbr) in {tpl_pk}
"""
print("del_sql ",del_sql)
execute_bq_query(del_sql)
df_update['inserted_at'] = datetime.now()
bq_values_insert(df_update, f"{table_name}", 2)


DELETE FROM biteam.sync_dms_pda_sod
WHERE
DATE(crtd_datetime) in ('2022-05-13', '2022-05-12', '2022-05-11')
AND concat(branchid,ordernbr) in ('MR0001DH0-0522-02211', 'MR0001DH0-0522-02212', 'MR0001DH0-0522-02213', 'MR0001DH0-0522-02214', 'MR0001DH0-0522-02219', 'MR0001DH0-0522-02223', 'MR0001DH0-0522-02224', 'MR0001DH0-0522-02227', 'MR0001DH0-0522-02236', 'MR0001DH0-0522-02241', 'MR0001DH0-0522-02242', 'MR0015DH5-0522-00862', 'MR0010DH1-0522-01603', 'MR0010DH1-0522-01621', 'MR0010DH1-0522-01623', 'MR0012DH2-0522-00427', 'MR0001DH0-0522-01992', 'MR0001DH0-0522-01993', 'MR0001DH0-0522-02020', 'MR0013DH3-0522-00774', 'MR0013DH3-0522-00810', 'MR0013DH3-0522-00816', 'MR0013DH3-0522-00826', 'MR0013DH3-0522-00827', 'MR0013DH3-0522-00828', 'MR0016DH6-0522-00857', 'MR0016DH6-0522-00883', 'MR0016DH6-0522-00887', 'MR0010DH1-0522-01356')



In [17]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> insert >> update